#Downloads and Imports




In [ ]:
pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.2/34.2 MB 25.1 MB/s eta 0:00:00


In [ ]:
pip install selfies

In [ ]:
pip install datamol

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.4/495.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.3 MB/s eta 0:00:00


In [ ]:
pip install scikit-learn

In [ ]:
!pip install --verbose --no-cache-dir torch-scatter
!pip install --verbose --no-cache-dir torch-sparse
!pip install --verbose --no-cache-dir torch-cluster
!pip install torch-geometric
!pip install tensorboardX
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 3.1 MB/s eta 0:00:00
  Running command python setup.py egg_info
  Preparing metadata (setup.py) ... canceled
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = r

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load/import libraries
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import PandasTools
import datamol as dm
import selfies as sf

from tensorflow import keras

dm.disable_rdkit_log()

#Preprocessing

In [ ]:
# loading the datasets in

toxcast = pd.read_csv('/content/drive/MyDrive/Helen Tran - Science Research 2024-2025/Data/toxcast_data.csv')

## Removing Invalid Values

In [ ]:
# removing nans from toxcast
smiles_data = toxcast['smiles']
feature_data = toxcast.drop(columns = ['smiles'])
feature_data_clean = feature_data.notna().astype(float)
smiles_clean = smiles_data.loc[feature_data_clean.index]
toxcast_clean = feature_data_clean.copy()
toxcast_clean['smiles'] = smiles_clean

toxcast_clean = toxcast_clean.merge(smiles_clean, on = 'smiles')
print(toxcast_clean)

      ACEA_T47D_80hr_Negative  ACEA_T47D_80hr_Positive  \
0                         1.0                      1.0   
1                         0.0                      0.0   
2                         0.0                      0.0   
3                         1.0                      1.0   
4                         1.0                      1.0   
...                       ...                      ...   
8802                      0.0                      0.0   
8803                      0.0                      0.0   
8804                      0.0                      0.0   
8805                      0.0                      0.0   
8806                      0.0                      0.0   

      APR_HepG2_CellCycleArrest_24h_dn  APR_HepG2_CellCycleArrest_24h_up  \
0                                  0.0                               0.0   
1                                  0.0                               0.0   
2                                  0.0                               0.0   

## Molecular Standardization

In [ ]:
# Using datamol + RDKit for molecular standardization for preprocessing, toxcast first

smiles_column = "smiles"

def _preprocess(row):
    try:
      mol = dm.to_mol(row[smiles_column], ordered=True)
      mol = dm.fix_mol(mol)
      mol = dm.sanitize_mol(mol, sanifix=True, charge_neutral=False)
      mol = dm.standardize_mol(
          mol,
          disconnect_metals = False,
          normalize = True,
          reionize = True,
          uncharge = False,
          stereo = True,
      )
      row["standard_smiles"] = dm.standardize_smiles(dm.to_smiles(mol))
      row["standard_smiles"] = dm.standardize_smiles(dm.to_smiles(mol))
      row["selfies"] = dm.to_selfies(mol)
      row["selfies"] = dm.to_selfies(mol)
      row["inchi"] = dm.to_inchi(mol)
      row["inchikey"] = dm.to_inchikey(mol)
      return row
    except:
        print("An exception occured.")


toxcast_standardized = toxcast_clean.apply(_preprocess, axis = 1)
print(toxcast_standardized)

An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An exception occured.
An excepti

# TTV Split

In [ ]:
#remove nans, since molecular standardization added some
# merged_sets_clean = merged_sets.dropna()
# print(merged_sets_clean)
toxcast_standardized_clean = toxcast_standardized.dropna()
print(toxcast_standardized_clean)

      ACEA_T47D_80hr_Negative  ACEA_T47D_80hr_Positive  \
0                         1.0                      1.0   
1                         0.0                      0.0   
2                         0.0                      0.0   
3                         1.0                      1.0   
4                         1.0                      1.0   
...                       ...                      ...   
8802                      0.0                      0.0   
8803                      0.0                      0.0   
8804                      0.0                      0.0   
8805                      0.0                      0.0   
8806                      0.0                      0.0   

      APR_HepG2_CellCycleArrest_24h_dn  APR_HepG2_CellCycleArrest_24h_up  \
0                                  0.0                               0.0   
1                                  0.0                               0.0   
2                                  0.0                               0.0   

In [ ]:
# merging selfies columns together
# merged_sets_selfies = merged_sets_clean['selfies']
# merged_sets_selfies = merged_sets_selfies.copy()
# merged_sets_selfies.columns = ['selfies1', 'selfies2']
# merged_sets_selfies['selfies'] = merged_sets_selfies['selfies1'] + merged_sets_selfies['selfies2']
# merged_sets_selfies_1 = merged_sets_selfies.drop(['selfies1', 'selfies2'], axis = 1)

#Feature Selection

In [ ]:
print(toxcast_standardized_clean)

      ACEA_T47D_80hr_Negative  ACEA_T47D_80hr_Positive  \
0                         1.0                      1.0   
1                         0.0                      0.0   
2                         0.0                      0.0   
3                         1.0                      1.0   
4                         1.0                      1.0   
...                       ...                      ...   
8802                      0.0                      0.0   
8803                      0.0                      0.0   
8804                      0.0                      0.0   
8805                      0.0                      0.0   
8806                      0.0                      0.0   

      APR_HepG2_CellCycleArrest_24h_dn  APR_HepG2_CellCycleArrest_24h_up  \
0                                  0.0                               0.0   
1                                  0.0                               0.0   
2                                  0.0                               0.0   

In [ ]:
#defining X and y
# X = merged_sets_selfies_1['selfies']
# y = merged_sets_clean['APR_HepG2_CellCycleArrest_24h_dn']

X = toxcast_standardized_clean['selfies']
y = toxcast_standardized_clean['APR_HepG2_CellCycleArrest_24h_dn']

In [ ]:
#train-test-validation split, 80/20 train-test split, 75/25 train-validation split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state = 42)

# One-Hot Encoding

In [ ]:
#encoding training dataframe first

# Example DataFrame with SELFIES column
df_selfies_train = X_train

# Define a function to convert SELFIES string to a one-hot encoding vector
def selfies_to_onehot(selfies_str):
    # List of possible characters in SELFIES
    alphabet = sf.get_alphabet_from_selfies(df_selfies_train)
    alphabet.add("[nop]")
    alphabet = list(sorted(alphabet))

    # Create a one-hot encoded vector
    onehot = np.zeros(len(alphabet), dtype = int)
    for i, char in enumerate(alphabet):
        if char in selfies_str:
            onehot[i] = 1
    return onehot

# Apply the function to convert all SELFIES to one-hot encoding
selfies_vectors = df_selfies_train.apply(selfies_to_onehot)

# Convert the list of one-hot encoded SELFIES to a numpy array
selfies_array_train = np.array(list(selfies_vectors))

# Print the one-hot encoded vectors
print(selfies_array_train)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]


In [ ]:
#encoding validation set

# Example DataFrame with SELFIES column
df_selfies_val = X_val

# Define a function to convert SELFIES string to a one-hot encoding vector
def selfies_to_onehot(selfies_str):
    # List of possible characters in SELFIES
    alphabet = sf.get_alphabet_from_selfies(df_selfies_val)
    alphabet.add("[nop]")
    alphabet = list(sorted(alphabet))

    # Create a one-hot encoded vector
    onehot = np.zeros(len(alphabet), dtype = int)
    for i, char in enumerate(alphabet):
        if char in selfies_str:
            onehot[i] = 1
    return onehot

# Apply the function to convert all SELFIES to one-hot encoding
selfies_vectors = df_selfies_val.apply(selfies_to_onehot)

# Convert the list of one-hot encoded SELFIES to a numpy array
selfies_array_val = np.array(list(selfies_vectors))

# Print the one-hot encoded vectors
print(selfies_array_val)

[[1 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]


In [ ]:
#encoding test set
# Example DataFrame with SELFIES column
df_selfies_test = X_test

# Define a function to convert SELFIES string to a one-hot encoding vector
def selfies_to_onehot(selfies_str):
    # List of possible characters in SELFIES
    alphabet = sf.get_alphabet_from_selfies(df_selfies_test)
    alphabet.add("[nop]")
    alphabet = list(sorted(alphabet))

    # Create a one-hot encoded vector
    onehot = np.zeros(len(alphabet), dtype = int)
    for i, char in enumerate(alphabet):
        if char in selfies_str:
            onehot[i] = 1
    return onehot

# Apply the function to convert all SELFIES to one-hot encoding
selfies_vectors = df_selfies_test.apply(selfies_to_onehot)

# Convert the list of one-hot encoded SELFIES to a numpy array
selfies_array_test = np.array(list(selfies_vectors))

# Print the one-hot encoded vectors
print(selfies_array_test)

[[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


# Conversion to Torch Tensors

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Convert to torch tensor (training)
X_tox_tensor_train = torch.from_numpy(np.asarray(selfies_array_train))
y_tox_tensor_train = torch.from_numpy(np.asarray(y_train))

#Convert to torch tensor (val)
X_tox_tensor_val = torch.from_numpy(np.asarray(selfies_array_val))
y_tox_tensor_val = torch.from_numpy(np.asarray(y_val))

#Convert to torch tensor (test)
X_tox_tensor_test = torch.from_numpy(np.asarray(selfies_array_test))
y_tox_tensor_test = torch.from_numpy(np.asarray(y_test))

#Graph Neural Network (Model Training)

In [ ]:
#type assignment
X_tox_tensor_train = X_tox_tensor_train.type(torch.FloatTensor)
y_tox_tensor_train = y_tox_tensor_train.type(torch.FloatTensor)

In [ ]:
from tensorflow.keras import layers, regularizers, models
import torch.nn.functional as F
model = tf.keras.Sequential([
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(10, activation='softmax')
])

In [ ]:
print(X_tox_tensor_train.shape)
print(y_tox_tensor_train.shape)

torch.Size([5147, 155])
torch.Size([5147])


In [ ]:
# inputs/outputs
x_data = X_tox_tensor_train
y_true = y_tox_tensor_train

class Toxicity(nn.Module):
    def __init__(self):
        super(Toxicity, self).__init__()
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(155, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

# create model, define optimizer and loss function
model = Toxicity()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop (simplified)
for epoch in range(100):  # 100 epochs for training
    optimizer.zero_grad()  # Clear previous gradients

    # Forward pass
    y_pred = model(x_data)  # Get the model's output
    # assign thresholds for classes here

    # Compute loss
    loss = criterion(y_pred, y_true) # Compare output with true values

    # Backpropagation (compute gradients)
    loss.backward()

    # Update weights
    optimizer.step()

    # Print loss at each epoch
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([5147])) that is different to the input size (torch.Size([5147, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Loss: 0.1359769105911255
Epoch 2, Loss: 0.1235448345541954
Epoch 3, Loss: 0.11715206503868103
Epoch 4, Loss: 0.10918436199426651
Epoch 5, Loss: 0.1072608232498169
Epoch 6, Loss: 0.10756991058588028
Epoch 7, Loss: 0.10795073211193085
Epoch 8, Loss: 0.10783324390649796
Epoch 9, Loss: 0.10732831805944443
Epoch 10, Loss: 0.10676855593919754
Epoch 11, Loss: 0.10626233369112015
Epoch 12, Loss: 0.10580284148454666
Epoch 13, Loss: 0.10549735277891159
Epoch 14, Loss: 0.10523463785648346
Epoch 15, Loss: 0.10511830449104309
Epoch 16, Loss: 0.10508672893047333
Epoch 17, Loss: 0.10509244352579117
Epoch 18, Loss: 0.10514046251773834
Epoch 19, Loss: 0.10516731441020966
Epoch 20, Loss: 0.10516950488090515
Epoch 21, Loss: 0.10513879358768463
Epoch 22, Loss: 0.1050996333360672
Epoch 23, Loss: 0.10502813756465912
Epoch 24, Loss: 0.1049407348036766
Epoch 25, Loss: 0.10487464815378189
Epoch 26, Loss: 0.10481512546539307
Epoch 27, Loss: 0.10477756708860397
Epoch 28, Loss: 0.10474827885627747
Epoch 

# Model Evaluation

# New Section

In [ ]:
#metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve

In [ ]:
#detach to numpy
y_true_detach = y_true.detach().numpy()
y_pred_detach = y_pred.detach().numpy()

In [ ]:
#convert to binary
y_pred_binary = (y_pred_detach >= 0.5).astype(int)
print(y_pred_binary)

[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


In [ ]:
#accuracy
accuracy = accuracy_score(y_true_detach, y_pred_binary, normalize = True)
print(accuracy)

#confusion matrix
confusion_matrix = confusion_matrix(y_true_detach, y_pred_binary)
print(confusion_matrix)

#precision score
precision_score = precision_score(y_true_detach, y_pred_binary, zero_division = 0)
print(precision_score)

#recall score
recall_score = recall_score(y_true_detach, y_pred_binary)
print(recall_score)

#f1 score
f1_score = f1_score(y_true_detach, y_pred_binary)
print(f1_score)

#roc_auc score
roc_auc_score = roc_auc_score(y_true_detach, y_pred_detach)
print(roc_auc_score)

# #roc curve
# fpr, tpr, thresholds = roc_curve(y_true_detach, y_pred_detach)

0.8812900718865359
[[4536    0]
 [ 611    0]]
0.0
0.0
0.0
0.48402505361725223
